# Gráfico de Waffle, Nube de Palabras y Regresión

## Objetivos

Después de completar este laboratorio será capas de:

*   Crear gráficos de Waffle y Nube de Palabras
*   Crear gráficos de Regresión con la biblioteca Seaborn


## Tabla de Contenidos

1.  [Explorando Conjunto de Datos con *pandas* y *Matplotlib*](#0)
2.  [Descarga y Preparación de Datos](#2)
3.  [Visualización de datos usando Matplotlib](#4)
4.  [Grafico de Waffle](#6)
5.  [Gráfico de Nube de Palabras](#8)
6.  [Gráfico de Regresión](#10)


# Explorando Conjunto de Datos con *pandas* y *Matplotlib*<a id="0"></a>

Kit de herramientas: El curso se basa en gran medida en [**pandas**](http://pandas.pydata.org) y [**Numpy**](http://www.numpy.org/) para la manipulación, el análisis y la visualización de datos. La biblioteca principal de visualización que se está explorando en el curso es [**Matplotlib**](http://matplotlib.org/).

Conjunto de datos: Inmigración a Canada desde 1980 a 2013 - [International migration flows to and from selected countries - The 2015 revision](http://www.un.org/en/development/desa/population/migration/data/empirical2/migrationflows.asp) desde el sitio web de las Naciones Unidas.

El conjunto de datos contiene datos anuales sobre los flujos de migrantes internacionales registrados por los países de destino. Los datos presentan tanto las entradas como las salidas según el lugar de nacimiento, ciudadanía o lugar de residencia anterior/próxima tanto para extranjeros como para nacionales. Este laboratorio se concentra en los datos de inmigración canadiense.

# Descarga y Preparación de Datos <a id="2"></a>


Lo primero que hay que realizar es la instalación de **openpyxl** (antiguamente conocido como **xlrd**) que es un módulo que **pandas** requiere para leer archivos Excel.

In [ ]:
!pip install openpyxl

A continuación, se procederá a cargar los módulos claves para hacer análisis de datos **pandas** y **numpy**.


In [ ]:
import numpy as np  # util para computación científica en Python
import pandas as pd # biblioteca que contienen la estructura de datos de uso principal
from PIL import Image # convertir imágenes en matrices

Ahora, se carga el conjunto de datos con los datos de la inmigración en Canadá usando el método de **pandas** `read_excel()`.


In [ ]:
df_can = pd.read_excel(
    'datos/Canada.xlsx',
    sheet_name='Canada by Citizenship',
    skiprows=range(20),
    skipfooter=2)

print('Datos leídos y cargados en un dataframe de pandas!')

Mostrar las primeras 5 filas del conjunto de datos usando la función `head()`.

In [ ]:
df_can.head()

Descubrir cuantas entradas hay en el conjunto de datos.


In [ ]:
# dimensión del dataframe (filas, columnas)
df_can.shape  

Limpiar datos. Se realizarán algunas modificaciones al conjunto de datos original para que sea más fácil crear visualizaciones. La explicación de estas modificaciones están detalladas en los laboratorios anteriores *"Introducción a Matplotlib y Gráficos de línea"* y *"Gráficos de Área, Barras e Histogramas"*.


In [ ]:
# limpiar el conjunto de datos para eliminar columnas innecesarias (por ejemplo, REG) 
df_can.drop(['AREA', 'REG', 'DEV', 'Type', 'Coverage'], axis=1, inplace=True)

# cambiar el nombre de algunas de las columnas para que tengan sentido
df_can.rename(columns={'OdName':'Country', 'AreaName':'Continent','RegName':'Region'}, inplace=True)

# Para mantener la coherencia, asegúrese de que todas las etiquetas de las columnas sean de tipo cadena de caracteres o string
df_can.columns = list(map(str, df_can.columns))

# Establecer el nombre del país como índice. Esto es útil para buscar rápidamente países utilizando el método .loc
df_can.set_index('Country', inplace=True)

# Agregar la columna total
df_can['Total'] = df_can.sum(axis=1, numeric_only=True)

# finalmente, se crea una lista de años desde 1980 - 2013, esto será útil cuando comencemos a graficar los datos
years = list(map(str, range(1980, 2014)))
print('Dimensión de los datos:', df_can.shape)

# Visualización de datos usando Matplotlib<a id="4"></a>


Importar la biblioteca `matplotlib`.


In [ ]:
# se hará uso del despliegue de figuras en línea o dentro de Jupyter Notebook
%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches # necesario para los gráficos de waffle

mpl.style.use('ggplot')  # opcional: para estilo similar a ggplot

# verificar la versión de Matplotlib
print('Matplotlib version: ', mpl.__version__)

# Gráfico de Waffle <a id="6"></a>

Un *Gráfico de waffle* es una visualización interesante que normalmente se crea para mostrar el progreso hacia las metas. Por lo general, es una opción eficaz cuando se intenta agregar funciones de visualización interesantes a un objeto visual que consta principalmente de celdas, como un tablero de Excel.

Se retomará el estudio del caso anterior sobre Dinamarca, Noruega y Suecia.

In [ ]:
# crear un nuevo dataframe con los daros de los tres países 
df_dsn = df_can.loc[['Denmark', 'Norway', 'Sweden'], :]

# Visualizar los datos del dataframe
df_dsn

Desafortunadamente, los *gráficos de waffle* no están integrados en ninguna de las bibliotecas de visualización de Python. Por esto, se enseñará a crearlos desde cero.


**Paso 1.** El primer paso para crear un gráfico de waffle es determinar la proporción de cada categoría con respecto al total.

In [ ]:
# calcular la proporción de cada categoría con respecto al total
valores_total = df_dsn['Total'].sum()
proporcion_categorias = df_dsn['Total'] / valores_total

# mostrar proporción
pd.DataFrame({"Proporción Categoría ": proporcion_categorias})

**Paso 2.** El segundo paso es definir el tamaño total del *gráfico de waffle*.


In [ ]:
ancho = 40 # ancho del gráfico
alto = 10  # alto del gráfico

num_total_cuadros = ancho * alto # número total de cuadrados

print(f'Número total de cuadrados: {num_total_cuadros}.')

**Paso 3.** El tercer paso es usar la proporción de cada categoría para determinar el número respectivo de cuadrados que le corresponde a cada una.


In [ ]:
# calcular el número de cuadrados para cada categoría
cuadros_por_categoria = (proporcion_categorias * num_total_cuadros).round().astype(int)

# mostrar el número de cuadrados por categoría
pd.DataFrame({"Número de cuadrados": cuadros_por_categoria})

Según las proporciones calculadas, Dinamarca ocupará 129 cuadrados del *gráfico de waffle*, Noruega ocupará 77 cuadrados y Suecia ocupará 194 cuadrados.

**Paso 4.** El cuarto paso es crear una matriz que se parezca a un *gráfico de waffle* y llenarlo.


In [ ]:
# inicializar el gráfico de waffle con una matriz vacía
grafico_waffle = np.zeros((alto, ancho), dtype = np.uint)

# definir índices para recorrer el gráfico de waffle
categoria_indice = 0
cuadrado_indice = 0

# poblar el gráfico de waffle
for col in range(ancho):
    for fil in range(alto):
        cuadrado_indice += 1

        #  si el número de cuadrados poblado para la categoría actual es igual al numero de cuadrados asignados ...
        if cuadrado_indice > sum(cuadros_por_categoria[0:categoria_indice]):
            # ...pasa a la siguiente categoria
            categoria_indice += 1       
            
        # configurar el valor de la clases a entero, el cual se incrementa con la clase
        grafico_waffle[fil, col] = categoria_indice
        
print ('Gráfico de Waffle poblado!')

Observar cómo se ve la matriz.

In [ ]:
print(grafico_waffle)

Como era de esperar, la matriz consta de tres categorías y el número total de instancias de cada categoría coincide con el número total de mosaicos asignados a cada categoría.


**Paso 5.** Asigne la matriz del *gráfico de waffle* a un objeto visual.

In [ ]:
# crear un nuevo objeto del tipo figura
figura = plt.figure()

# usar matshow para mostrar el gráfico de waffle
mapacolores = plt.cm.coolwarm
plt.matshow(grafico_waffle, cmap=mapacolores)
#plt.rcParams['axes.grid'] = False
#plt.colorbar()
plt.show()

**Paso 6.** Embellecer el gráfico.

In [ ]:
# crear un nuevo objeto del tipo figura
figura = plt.figure()

# usar matshow para mostrar el gráfico de waffle
mapacolores = plt.cm.coolwarm
plt.matshow(grafico_waffle, cmap=mapacolores)
#plt.rcParams['axes.grid'] = False
#plt.colorbar()

# obtener los ejes
ax = plt.gca()

# configurar la marca menor
ax.set_xticks(np.arange(-.5, (ancho), 1), minor=True)
ax.set_yticks(np.arange(-.5, (alto), 1), minor=True)
    
# agregar lineas a la grilla basado en las marcas menores
ax.grid(which='minor', color='w', linestyle='-', linewidth=2)

plt.xticks([])
plt.yticks([])
plt.show()

**Paso 7.** Crear una leyenda y agregarla al gráfico.


In [ ]:
# crear un nuevo objeto del tipo figura
figura = plt.figure()

# usar matshow para mostrar el gráfico de waffle
mapacolores = plt.cm.coolwarm
plt.matshow(grafico_waffle, cmap=mapacolores)
#plt.rcParams['axes.grid'] = False
#plt.colorbar()

# obtener los ejes
ax = plt.gca()

# configurar la marca menor
ax.set_xticks(np.arange(-.5, (ancho), 1), minor=True)
ax.set_yticks(np.arange(-.5, (alto), 1), minor=True)
    
# agregar lineas a la grilla basado en las marcas menores
ax.grid(which='minor', color='w', linestyle='-', linewidth=2)

plt.xticks([])
plt.yticks([])

# calcular la suma acumulativa de categorías individuales para hacer coincidir los esquemas de color entre el gráfico y la leyenda
valores_suma_acumulativa = np.cumsum(df_dsn['Total'])
valor_total = valores_suma_acumulativa[len(valores_suma_acumulativa) - 1]

# crear leyenda
leyenda_manejadores = []
for i, categoria in enumerate(df_dsn.index.values):
    etiqueta_string = categoria + ' (' + str(df_dsn['Total'][i]) + ')'
    color_valores = mapacolores(float(valores_suma_acumulativa[i])/valor_total)
    leyenda_manejadores.append(mpatches.Patch(color=color_valores, label=etiqueta_string))

# agregar la leyenda al gráfico
plt.legend(handles=leyenda_manejadores,
           loc='lower center', 
           ncol=len(df_dsn.index.values),
           bbox_to_anchor=(0., -0.2, 0.95, .1)
          )
plt.show()

Ahora, sería muy ineficiente repetir estos siete pasos cada vez que se desee crear un *gráfico de waffle*. Por este motivo, se van a juntar los 7 pasos en una función que se llame *crea_grafico_waffle*. Esta función tiene los siguientes parámetros de entrada:

> 1. **categorías**: Categorías o clases únicas en el dataframe.
> 2. **valores**: Valores correspondientes a categorías o clases.
> 3. **altura**: Altura definida del gráfico de waffle.
> 4. **ancho**: Ancho definido del gráfico de waffle.
> 5. **mapa de colores**: clase de mapa de colores
> 6. **signo_valores**: para que nuestra función sea más generalizable, agregaremos este parámetro para abordar los signos que podrían asociarse con un valor como %, $, etc. **signo_valores** tiene un valor predeterminado de cadena vacía.


In [ ]:
def crea_grafico_waffle(categorias, valores, alto, ancho, mapacolores, signo_valores=''):

    # calcular la proporción de cada categoría con respecto al total
    valores_total = sum(valores)
    proporcion_categorias = [(float(valor) / valores_total) for valor in valores]

    # calcula el número total de cuadrados
    num_total_cuadros = ancho * alto # número total de cuadrados
    print ('Número total de cuadrados:', num_total_cuadros)
    
    # calcula en número de cuadrados para cada categoría
    cuadros_por_categoria = [round(proporcion * num_total_cuadros) for proporcion in proporcion_categorias]

    # Imprimir el número de cuadrados por categoría
    for i, cuadrados in enumerate(cuadros_por_categoria):
        print (df_dsn.index.values[i] + ': ' + str(cuadrados))
    
    # inicializar el gráfico de waffle como una matriz vacía
    grafico_waffle = np.zeros((alto, ancho))

    # definir los indices para recorrer el gráfico de waffle
    categoria_indice = 0
    cuadrado_indice = 0

    # poblar el gráfico de waffle
    for col in range(ancho):
        for fil in range(alto):
            cuadrado_indice += 1

            #  si el número de cuadrados poblado para la categoría actual
            #  es igual al numero de cuadrados asignados ...
            if cuadrado_indice > sum(cuadros_por_categoria[0:categoria_indice]):
                # ...pasa a la siguiente categoria
                categoria_indice += 1       
            
            # configurar el valor de la clases a entero, el cual se incrementa con la clase
            grafico_waffle[fil, col] = categoria_indice

    # crear un nuevo objeto del tipo figura
    figura = plt.figure()

    # usar matshow para mostrar el gráfico de waffle
    mapacolores = plt.cm.coolwarm
    plt.matshow(grafico_waffle, cmap=mapacolores)
    #plt.rcParams['axes.grid'] = False
    #plt.colorbar()
       
    # obtener los ejes
    ax = plt.gca()

    # configurar la marca menor
    ax.set_xticks(np.arange(-.5, (ancho), 1), minor=True)
    ax.set_yticks(np.arange(-.5, (alto), 1), minor=True)
    
    # agregar lineas a la grilla basado en las marcas menores
    ax.grid(which='minor', color='w', linestyle='-', linewidth=2)

    plt.xticks([])
    plt.yticks([])

    # calcular la suma acumulativa de categorías individuales para hacer coincidir los esquemas de color entre el gráfico y la leyenda
    valores_suma_acumulativa = np.cumsum(df_dsn['Total'])
    valor_total = valores_suma_acumulativa[len(valores_suma_acumulativa) - 1]

    # crear leyenda
    leyenda_manejadores = []
    for i, categoria in enumerate(df_dsn.index.values):
        etiqueta_string = categoria + ' (' + str(df_dsn['Total'][i]) + ')'
        color_valores = mapacolores(float(valores_suma_acumulativa[i])/valor_total)
        leyenda_manejadores.append(mpatches.Patch(color=color_valores, label=etiqueta_string))

    # agregar la leyenda al gráfico
    plt.legend(handles=leyenda_manejadores,
            loc='lower center', 
            ncol=len(df_dsn.index.values),
            bbox_to_anchor=(0., -0.2, 0.95, .1)
            )
    plt.show()

Ahora, para crear un *gráfico de waffle*, todo lo que se debe hacer es llamar a la función `crea_grafico_waffle`. Primero, hay que definir los parámetros de entrada:

In [ ]:
ancho = 40 # ancho del gráfico
alto = 10 # alto del gráfico

categorias = df_dsn.index.values # categorías
valores = df_dsn['Total'] # valores de cada categoría

mapacolores = plt.cm.coolwarm # clase del mapa de colores

Finalmente se puede llamar a la función para crear el *gráfico de waffle*.


In [ ]:
crea_grafico_waffle(categorias, valores, alto, ancho, mapacolores)

Existen algunos proyectos con un paquete de Python para generar `gráficos de waffle` que se pueden probar, tales como:
- [PyWaffle](https://github.com/gyli/PyWaffle).
- [pywaffle](https://pypi.org/project/pywaffle/).

# Gráfico de Nube de Palabras <a id="8"></a>

Las *nubes de palabras* (también conocidas como nubes de texto o nubes de etiquetas) funcionan de manera simple: cuanto más aparece una palabra específica en una fuente de datos textuales (como un discurso, una publicación de blog o una base de datos), más grande y marcado aparece en la nube de palabras.


Existe un paquete en Python para generar *nubes de palabras*. El paquete, llamado `word_cloud`, fue desarrollado por **Andreas Mueller**. Puede obtener más información sobre este paquete siguiendo este [enlace](https://github.com/amueller/word_cloud/). Para filtrar las **palabras vacías** o **stop words** en español, inglés u otro idioma se va a utilizar el paquete `NLTK` (Natural Language Toolkit). Puede obtener más información sobre este paquete siguiendo este [enlace](https://www.nltk.org/index.html).

Se usarán estos paquetes para aprender cómo generar una nube de palabras para un documento de texto dado.


Primero se debe instalar el paquete.


In [ ]:
# instalar wordcloud
!pip3 install wordcloud

# instalar NLTK
!pip3 install NLTK

# importar los paquetes y su conjunto de stop words
import nltk
nltk.download('stopwords')
from wordcloud import WordCloud
from nltk.corpus import stopwords

print ('Wordcloud  y NLTK están instalados e importados!')

Las *nubes de palabras* se usan comúnmente para realizar análisis de alto nivel y visualización de datos de texto. En consecuencia, se dejará, momentáneamente, de trabajar con el conjunto de datos de inmigración y se trabajará con un ejemplo que involucra el análisis de datos de texto. A continuación se leerá un archivo *.txt* de algún libro.


In [ ]:
fd=open("datos/Texto_El_Arbol_De_La_Colina.txt","r")
texto_libro=fd.read()


A continuación, se usarán las palabras vacías que se importan de `NLTK`. Se usa la función *set* para eliminar palabras vacías redundantes.


In [ ]:
stop_words_sp = set(stopwords.words('spanish'))
stop_words_en = set(stopwords.words('english'))
stop_words = stop_words_sp | stop_words_en

Se creará un objeto de nube de palabras y se generará una nube de palabras. Para simplificar, se generará una nube de palabras usando solo las primeras 2000 palabras del libro.


In [ ]:
# instanciar un objeto de nube de palabras
texto_wc = WordCloud(
    background_color='white',
    max_words=2000,
    stopwords=stop_words
)

# generar la nube de palabras
texto_wc.generate(texto_libro)

Ahora que se ha creado la *nube de palabras* se puede visualizar.


In [ ]:
# desplegar la nube de palabras
plt.imshow(texto_wc, interpolation='bilinear')
plt.axis('off')
plt.show()

Entonces, en las primeras 2000 palabras de la novela, las palabras más comunes son **árbol**, **colina**, **lugar**, **aquel**, etc. Se redimensionará la nube para que podamos ver un poco mejor las palabras menos frecuentes.


In [ ]:
fig = plt.figure(figsize=(14, 18))

# visualizar la nube
plt.imshow(texto_wc, interpolation='bilinear')
plt.axis('off')
plt.show()

Sin embargo, algunas palabras no son realmente una palabra informativa. Así que se debiera de agregar al conjunto de palabras vacías y regenerar la nube.

In [ ]:
stop_words.add('tan') # agrega una palabra a las palabras vacías

# re-generar la nube de palabras
texto_wc.generate(texto_libro)

# desplegar la nube
fig = plt.figure(figsize=(14, 18))

plt.imshow(texto_wc, interpolation='bilinear')
plt.axis('off')
plt.show()

Este tipo de gráfica es interesante y descriptivo, sin embargo, el paquete `word_cloud` permite superponer las palabras en una máscara de cualquier forma. Se usará una máscara de una silueta de un árbol

In [ ]:
# Guardar la máscara en texto_mascara
texto_mascara = np.array(Image.open('imagenes/Arbol_mascara2.jpg'))

Mostrar cómo se ve la máscara.


In [ ]:
figura_wc = plt.figure(figsize=(14, 18))

plt.imshow(texto_mascara, cmap=plt.cm.gray, interpolation='bilinear')
plt.axis('off')
plt.show()

Dar forma a la *nube de palabras* de acuerdo con la máscara es sencillo usando el paquete `word_cloud`. Para simplificar, se continuará usando las primeras 2000 palabras del libro.


In [ ]:
#  instanciar un objeto de nube de palabras
texto_wc = WordCloud(background_color='white', max_words=2000, mask=texto_mascara, stopwords=stop_words)

# generar la nube de palabras
texto_wc.generate(texto_libro)

# desplegar la nube de palabras
figura_wc = plt.figure(figsize=(14, 18))

plt.imshow(texto_wc, interpolation='bilinear')
plt.axis('off')
plt.show()

Desafortunadamente, los datos de inmigración no tienen datos de texto. Sin embargo, se puede generar datos de texto de muestra a partir del conjunto de datos de inmigración, así se generarán datos de texto de 90 palabras.


Recordar la estructura del dataframe.


In [ ]:
df_can.head()

Recordar el total de los datos de inmigración entre 1980 y 2013.


In [ ]:
inmigracion_total = df_can['Total'].sum()
inmigracion_total

Se usarán países con nombres de una sola palabra, se multiplicará el nombre de cada país según cuánto contribuyen a la inmigración total.


In [ ]:
num_palabras = 90
palabras_string = ''
for pais in df_can.index.values:
    # verificar si en nombre del país tiene solo una palabra
    if pais.count(" ") == 0:
        repetir_num_veces = int(df_can.loc[pais, 'Total'] / inmigracion_total * num_palabras)
        palabras_string = palabras_string + ((pais + ' ') * repetir_num_veces)

# mostrar el texto generado
palabras_string

Como el texto generado con contiene palabras vacías, no es necesarios filtrarlas del texto a ser procesado para crear la nube de palabras.


In [ ]:
# crear la nube de palabras
nube_de_palabras = WordCloud(background_color='white').generate(palabras_string)

print('Nube de palabras creada!')

In [ ]:
# desplegar la nube
plt.figure(figsize=(14, 18))

plt.imshow(nube_de_palabras, interpolation='bilinear')
plt.axis('off')
plt.show()

De acuerdo con la nube de palabras anterior, parece que la mayoría de las personas que inmigraron procedían de alguno de los 15 países que se muestran en la nube de palabras. Una buena máscara que se podría aplicar es el mapa de Canadá y superponer la nube de palabras sobre este mapa.

# Gráfico de Regresión <a id="10"></a>

> Seaborn es una biblioteca de visualización de Python basada en matplotlib. Proporciona una interfaz de alto nivel para dibujar atractivos gráficos estadísticos. Puede obtener más información sobre *seaborn* siguiendo este [enlace](https://seaborn.pydata.org/) y más sobre gráficos de regresión *seaborn* siguiendo este [enlace](http://seaborn.pydata.org/generated/seaborn.regplot.html).


En el laboratorio *Gráficos de Círculo, Caja, Dispersión y Burbuja*, se vio cómo crear un diagrama de dispersión y luego ajustar una línea de regresión. Se necesitaron alrededor de \~20 líneas de código para crear el diagrama de dispersión junto con el ajuste de regresión. En esta sección final, se explorará *seaborn* y se verá cuán eficiente es crear líneas de regresión y ajustes usando esta biblioteca.


Lo primero que hay que hacer es instalar la biblioteca *seaborn*


In [ ]:
# instalar seaborn
!pip3 install seaborn

# importar la biblioteca
import seaborn as sns

print('Seaborn instalado e importado!')

Crear un nuevo marco de datos que almacene la cantidad total de inmigrantes que llegaron en Canadá por año desde 1980 hasta 2013.


In [ ]:
# usar el método sum() para obtener la población total por año
df_total = pd.DataFrame(df_can[years].sum(axis=0))

# cambiar los años al tipo float (útil para la regresión más adelante)
df_total.index = map(float, df_total.index)

# restablecer el índice para volver a colocarlo como una columna en el marco de datos df_total
df_total.reset_index(inplace=True)

# renonmbrar las columnas
df_total.columns = ['Año', 'Total']

# mostrar el dataframe final
df_total.head()

Con *seaborn*, generar un diagrama de regresión es tan simple como llamar a la función **regplot**.


In [ ]:
sns.regplot(x='Año', y='Total', data=df_total)

También se puede personalizar el color del diagrama de dispersión y la línea de regresión. Vamos a cambiar el color a verde.


In [ ]:
sns.regplot(x='Año', y='Total', data=df_total, color='green')
plt.show()

Siempre puede personalizar la forma del marcador, así que en lugar de marcadores circulares, se usará `+`.


In [ ]:
ax = sns.regplot(x='Año', y='Total', data=df_total, color='green', marker='+')
plt.show()

Se modificará el tamaño de la trama para que sea más atractiva a la vista.


In [ ]:
plt.figure(figsize=(15, 10))
sns.regplot(x='Año', y='Total', data=df_total, color='green', marker='+')
plt.show()

Se aumentará el tamaño de los marcadores para que coincidan con el nuevo tamaño de la figura, y agreguemos un título y etiquetas a los ejes x e y.


In [ ]:
plt.figure(figsize=(15, 10))
ax = sns.regplot(x='Año', y='Total', data=df_total, color='green', marker='+', scatter_kws={'s': 200})

ax.set(xlabel='Año', ylabel='Total Inmigrantes') # agregar etiquetas a los ejes x e y
ax.set_title('Inmigración Total a Canada entre 1980 - 2013') # agregar titulo
plt.show()

Por último, aumentar el tamaño de las letras de las marcas, el título y las etiquetas x e y para que lean bien.


In [ ]:
plt.figure(figsize=(15, 10))

sns.set(font_scale=1.7)

ax = sns.regplot(x='Año', y='Total', data=df_total, color='green', marker='+', scatter_kws={'s': 200})
ax.set(xlabel='Año', ylabel='Total Inmigrantes')
ax.set_title('Inmigración Total a Canada entre 1980 - 2013')
plt.show()

Un diagrama de dispersión completo con un ajuste de regresión con solo 5 líneas de código.

También se puede cambiar fácilmente el estilo a un fondo blanco liso.


In [ ]:
plt.figure(figsize=(15, 10))

sns.set(font_scale=1.7)
sns.set_style('ticks')  # cambiar el fondo a un color blanco

ax = sns.regplot(x='Año', y='Total', data=df_total, color='green', marker='+', scatter_kws={'s': 200})
ax.set(xlabel='Año', ylabel='Total Inmigrantes')
ax.set_title('Inmigraciónn Total a Canada enter 1980 - 2013')
plt.show()

O a un fondo blanco con líneas de marcas.


In [ ]:
plt.figure(figsize=(15, 10))

sns.set(font_scale=1.7)
sns.set_style('whitegrid')

ax = sns.regplot(x='Año', y='Total', data=df_total, color='green', marker='+', scatter_kws={'s': 200})
ax.set(xlabel='Año', ylabel='Total Inmigrantes')
ax.set_title('Inmigraciónn Total a Canada enter 1980 - 2013')
plt.show()

**Pregunta**: Use *seaborn* para crear un diagrama de dispersión con una línea de regresión para visualizar la inmigración total de Dinamarca, Suecia y Noruega a Canadá desde 1980 hasta 2013.


In [ ]:
### Escriba su código a continuación y presione Shift+Enter para ejecutarlo



Doble click **aquí** para ver la solución

<!--

#La respuesta correcta es:   
    
# crear el dataframe df_paises
df_paises = df_can.loc[['Denmark', 'Norway', 'Sweden'], years].transpose()

# crear df_total sumando los valores para cada país por cada año
df_total = pd.DataFrame(df_paises.sum(axis=1))

# restaurar el índice a su lugar
df_total.reset_index(inplace=True)

# renombrar las columnas
df_total.columns = ['Año', 'Total']

# cambiar la columna año de string a entero para crear el gráfico de dispersión
df_total['Año'] = df_total['Año'].astype(int)

# definir el tamaño de la figura
plt.figure(figsize=(15, 10))

# definir el estilo del fondo y tamaño de letra
sns.set(font_scale=1.5)
sns.set_style('whitegrid')

# crear elgráfico y las etiquetas del título y los ejes
ax = sns.regplot(x='Año', y='Total', data=df_total, color='green', marker='+', scatter_kws={'s': 200})
ax.set(xlabel='Año', ylabel='Total Inmigrantes')
ax.set_title('Inmigración Total desde Dinamarca, Suecia, y Noruega a Canadá entre 1980 - 2013')

-->